In [1]:
pip install groq

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 6.6 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from groq import Groq
from typing import List , Dict

In [3]:
client = Groq(
    api_key = "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"
)
model = "llama-3.3-70b-versatile"
token = client.api_key

In [4]:
from typing import List, Dict
from groq import Groq

class LawyerAgent:
    def __init__(self,
                 name: str,
                 system_prompt: str,
                 model: str = "llama-3.3-70b-versatile",
                 api_key: str = "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"):
        self.name = name
        self.system_prompt = system_prompt.strip()
        self.history: List[Dict[str, str]] = []      # list of {"role": ..., "content": ...}
        self.client = Groq(api_key=api_key)
        self.model = model

    def _format_prompt(self, user_msg: str) -> str:
        messages = [{"role": "system", "content": self.system_prompt}]
        messages.extend(self.history)
        messages.append({"role": "user", "content": user_msg})
        return messages

    def respond(self, user_msg: str, **gen_kwargs) -> str:
        messages = self._format_prompt(user_msg)
        response = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            temperature=0.7,
            max_tokens=512,
            **gen_kwargs
        )
        answer = response.choices[0].message.content.strip()
        self.history.append({"role": "user", "content": user_msg})
        self.history.append({"role": "assistant", "content": answer})
        return answer


In [5]:
from typing import List, Dict
from groq import Groq

class MainAgent:
    def __init__(self,
                 name: str,
                 system_prompt: str,
                 model: str = "llama-3.3-70b-versatile",
                 api_key: str = "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"):
        self.name = name
        self.system_prompt = system_prompt.strip()
        self.history: List[Dict[str, str]] = []      # list of {"role": ..., "content": ...}
        self.client = Groq(api_key=api_key)
        self.model = model

    def _format_prompt(self, user_msg: str) -> str:
        messages = [{"role": "system", "content": self.system_prompt}]
        messages.extend(self.history)
        messages.append({"role": "user", "content": user_msg})
        return messages

    def respond(self, user_msg: str, **gen_kwargs) -> str:
        messages = self._format_prompt(user_msg)
        response = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            temperature=0.7,
            max_tokens=512,
            **gen_kwargs
        )
        answer = response.choices[0].message.content.strip()
        self.history.append({"role": "user", "content": user_msg})
        self.history.append({"role": "assistant", "content": answer})
        return answer


In [6]:
DEFENSE_SYSTEM = """
You are **Alex Carter**, lead *defense counsel*.
Goals:
• Protect the constitutional rights of the defendant.
• Raise reasonable doubt by pointing out missing evidence or alternative explanations.
• Be respectful to the Court and to opposing counsel.
Style:
• Crisp, persuasive, grounded in precedent and facts provided.
• When citing precedent: give short case name + year (e.g., *Miranda v. Arizona* (1966)).
Ethics:
• Do not fabricate evidence; admit uncertainty when required.
"""

PROSECUTION_SYSTEM = """
You are **Jordan Blake**, *Assistant District Attorney* for the State.
Goals:
• Present the strongest good-faith case against the accused.
• Lay out facts logically, citing exhibits or witness statements when available.
• Anticipate and rebut common defense arguments.
Style:
• Formal but plain English; persuasive, with confident tone.
Ethics:
• Duty is to justice, not merely to win. Concede points when ethically required.
"""

DEFENDANT_PROMPT = """
        You are the defendant. Respond truthfully to all questions about your actions and intentions.
        Only speak when addressed. Admit what you know and clarify misunderstandings.
        """

PLAINTIFF_PROMPT = """
        You are the plaintiff. Clearly explain your grievance, the harm suffered, and what you seek from the court.
        Answer questions honestly and provide details about the incident.
        """


In [7]:
class DefendantAgent(MainAgent):
    def __init__(self,model: str,token: str,system_prompt: str):
        super().__init__("Defendant",system_prompt,model,token)
    def testify(self,question: str) -> str:
        return self.respond(f"As the defendant, answer: {question}")

In [8]:
class EyewitnessAgent(MainAgent):
    def __init__(self, name: str,witnessed_details:str, model:str,token: str):
        system_prompt = f"""
        You are an eyewitness named {name}.You witnessed the following: {witnessed_details}

        Your role is to:
        • Testify truthfully about what you saw, heard, or otherwise perceived
        • Answer questions from both lawyers clearly and accurately
        • Only speak about what you personally witnessed
        • Admit when you are uncertain or don't remember details

        Style:
        • Speak in first person ("I saw...")
        • Use descriptive but factual language
        • Maintain consistency in your account
        • Show appropriate emotional reactions based on what you witnessed
        """
        super().__init__(name, system_prompt, model, token)

    def testify(self)->str:
        return self.respond("Describe in detail what you witnessed regarding this case.")

In [9]:
class JudgeAgent(MainAgent):
    def __init__(self, model:str,token: str  , case_background: str , eyewitness_details:str):

        system_prompt = f"""
You are a constitutional law judge. Analyze using these criteria:

[Case Analysis Framework]
1. Evidence Admissibility:
   - Verify electronic evidence compliance with Section 65B, Indian Evidence Act [2]
   - Assess eyewitness reliability per IPC Section 9 & 27 [3]

2. Constitutional Compliance:
   - Due Process (Article 21)
   - Right Against Self-Incrimination (Article 20(3))
   - Fair Trial Protections (Article 14)

3. Burden of Proof:
   - Prosecution must prove guilt "beyond reasonable doubt" (Section 101 IEA)
   - Defense need only establish plausible doubt

[Input Data]
Case Background: {case_background}
Eyewitness Statement: {eyewitness_details}

[Decision Protocol]
Return 1 if ALL are true:
✓ Legally admissible evidence proves guilt conclusively
✓ No constitutional rights violations in evidence collection
✓ No alternative plausible explanations exist

Return 0 if ANY are true:
✓ Evidence obtained through unconstitutional means
✓ Eyewitness account shows "confidence inflation" [3]
✓ Chain of custody issues with digital evidence [2]
✓ Reasonable doubt persists after analysis

Return ONLY the digit 1 or 0. DO NOT explain. DO NOT say anything else.
If prosecution wins, output 1. If defense wins, output 0.
Your response must be a single character: 1 or 0.
"""


        super().__init__("Judge",system_prompt,model,token)
    def verdict(self) -> int:
        response = self.respond(
    "Based on all evidence and law, return ONLY 1 or 0. Do not explain. Do not say anything else."
)
        return self._parse_verdict(response)

    def _parse_verdict(self, text: str) -> int:
        text = text.strip().lower()
        if text == "1" or text.startswith("1"):
            return 1
        elif text == "0" or text.startswith("0"):
            return 0
        elif "guilty" in text:
            return 1
        elif "not guilty" in text:
            return 0
        else:
            return -1


In [10]:
class PlaintiffAgent(MainAgent):
    def __init__(self, model: str,token: str,system_prompt:str):
        super().__init__("Plaintiff",system_prompt,model,token)
    def statement(self) -> str:
        return self.respond("State your grievance and what you want from the court.")

In [11]:
case_background = "The State alleges that John Doe stole proprietary algorithms from his former employer and used them at a competitor. The charge is felony theft of trade secrets. No physical evidence shows direct copying, but server logs indicate large downloads two days before Doe resigned."

In [12]:
eyewitness_details = ""

In [ ]:
defense = LawyerAgent("Defense", DEFENSE_SYSTEM)
prosecution = LawyerAgent("Prosecution", PROSECUTION_SYSTEM)
defendant = DefendantAgent(model=model, token=token, system_prompt=DEFENDANT_PROMPT)
plaintiff = PlaintiffAgent(model=model, token=token, system_prompt=PLAINTIFF_PROMPT)
judge = JudgeAgent(
    model=model,
    token=token,
    case_background=case_background,
    eyewitness_details=eyewitness_details
)

# Opening Statements
print("\n=== OPENING STATEMENTS ===")
print("PROSECUTION:", prosecution.respond(f"Opening statement: {case_background}"))
print("DEFENSE:", defense.respond("Opening statement responding to prosecution"))

# Plaintiff Testimony
print("\n=== PLAINTIFF TESTIMONY ===")
print("PLAINTIFF:", plaintiff.statement())

# Eyewitness Testimony (if present)
if eyewitness_details.strip():
    eyewitness = EyewitnessAgent("Sairaj", eyewitness_details, model, token)
    print("\n=== EYEWITNESS TESTIMONY ===")
    print("EYEWITNESS:", eyewitness.testify())

# Defendant Testimony
print("\n=== DEFENDANT TESTIMONY ===")
print("DEFENDANT:", defendant.testify("Explain your side of the case"))

# Closing Statements
print("\n=== CLOSING STATEMENTS ===")
print("PROSECUTION:", prosecution.respond("Final closing argument"))
print("DEFENSE:", defense.respond("Final closing argument"))

# Judge's Verdict
print("\n=== JUDGE'S VERDICT ===")
verdict = judge.verdict()
print("Judge's Verdict:", verdict)
if verdict == 1:
    print("Judge's Verdict: 1 (Prosecution wins the case)")
elif verdict == 0:
    print("Judge's Verdict: 0 (Defense wins the case)")
else:
    print("Judge's Verdict: Unparseable response - requires human review.")

In [13]:
df = pd.read_csv("cases.csv")
df1 = df.head(50)

In [14]:
df1

,Unnamed: 0,id,text
0,0,1989_75,civil appellate jurisdiction civil appeal numb...
1,1,1959_102,civil appellate jurisdiction civil appeal numb...
2,2,1979_76,civil appellate jurisdiction civil appeal numb...
3,3,1991_124,civil appellate jurisdiction civil appeal numb...
4,4,1985_233,original jurisdiction writ petitions number. 8...
5,5,1984_73,civil appellate jurisdiction civil appeal\nnum...
6,6,1963_191,civil appellate jurisdiction civil appeal numb...
7,7,1992_1,criminal appellate jurisdiction criminal appea...
8,8,1963_285,\nsarkar j. \n\nwe think this appeal must be a...
9,9,1961_166,\nwanchoo j. \n\nthis is an appeal on a certif...


In [15]:
data = df1['text']
data

0     civil appellate jurisdiction civil appeal numb...
1     civil appellate jurisdiction civil appeal numb...
2     civil appellate jurisdiction civil appeal numb...
3     civil appellate jurisdiction civil appeal numb...
4     original jurisdiction writ petitions number. 8...
5     civil appellate jurisdiction civil appeal\nnum...
6     civil appellate jurisdiction civil appeal numb...
7     criminal appellate jurisdiction criminal appea...
8     \nsarkar j. \n\nwe think this appeal must be a...
9     \nwanchoo j. \n\nthis is an appeal on a certif...
10    \nshah j.\n\nthese are three appeals with cert...
11    civil appellate jurisdiction civil appeal numb...
12    civil appellate jurisdiction civil appeal numb...
13    civil appellate jurisdiction civil appeal numb...
14    criminal appellate jurisdiction criminal appea...
15    criminal appellate jurisdiction criminal appea...
16    civil appellate jurisdiction civil appeal numb...
17    civil appellate jurisdiction civil appeal 

In [16]:
data[0]


'civil appellate jurisdiction civil appeal number 1709\nof\n1988.\nfrom the judgment and order dated 9.10.1987 of t\nhe\ncentral administrative tribunal chandigarh in appln. t.\n--\n1055 of 1986.\nwith\n slp  civil  number 6998 of 1988 \n\nc. mahajan mrs. indu goswami c.v. subba rao\np.\nparmeshwaran for the appellant in c.a. number 1709 of 1988.\n\nk.d. namboodary for the petitioner in slp  civil  n\no. 6998 of 1988.\n\nm. ashri and mahabir singh for the respondents. the judgment of the companyrt was delivered by\n\njagannatha shetty j. the civil appeal by speci\nal\nleave and the companynected slp raise an important issue as\nto\nthe power of the central administration tribun\nal\n tribunal  to examine the adequacy of penalty awarded\nby\nthe companypetent authority to a government servant in discipl\ni-\nnary proceedings. short factual background is this\nparmanand--respondent in the appeal was a time keeper\nin\nbeas sutlej link project sundernagar. he was incharge\nof\npreparing t

In [ ]:
def run_trial(case_background, eyewitness_details=""):
    # Initialize agents
    defense = LawyerAgent("Defense", DEFENSE_SYSTEM)
    prosecution = LawyerAgent("Prosecution", PROSECUTION_SYSTEM)
    defendant = DefendantAgent(model=model, token=token, system_prompt=DEFENDANT_PROMPT)
    plaintiff = PlaintiffAgent(model=model, token=token, system_prompt=PLAINTIFF_PROMPT)
    judge = JudgeAgent(
        model=model,
        token=token,
        case_background=case_background,
        eyewitness_details=eyewitness_details if eyewitness_details is not None else ""
    )

    # Opening Statements
    print("\n=== OPENING STATEMENTS ===")
    print("PROSECUTION:", prosecution.respond(f"Opening statement: {case_background}"))
    print("DEFENSE:", defense.respond("Opening statement responding to prosecution"))

    # Plaintiff Testimony
    print("\n=== PLAINTIFF TESTIMONY ===")
    print("PLAINTIFF:", plaintiff.statement())

    # Eyewitness Testimony (if present)
    if eyewitness_details is not None and eyewitness_details.strip():
        eyewitness = EyewitnessAgent("Sairaj", eyewitness_details, model, token)
        print("\n=== EYEWITNESS TESTIMONY ===")
        print("EYEWITNESS:", eyewitness.testify())

    # Defendant Testimony
    print("\n=== DEFENDANT TESTIMONY ===")
    print("DEFENDANT:", defendant.testify("Explain your side of the case"))

    # Closing Statements
    print("\n=== CLOSING STATEMENTS ===")
    print("PROSECUTION:", prosecution.respond("Final closing argument"))
    print("DEFENSE:", defense.respond("Final closing argument"))

    # Judge's Verdict
    print("\n=== JUDGE'S VERDICT ===")
    verdict=judge.verdict()
    print("Judge's Verdict:",verdict)
    if verdict == 1:
        print("Judge's Verdict: 1 (Prosecution wins the case)")
    elif verdict == 0:
        print("Judge's Verdict: 0 (Defense wins the case)")
    else:
        print("Judge's Verdict: Unparseable response - requires human review.")

    return verdict

run_trial(case_background)


In [22]:
def run_trial(case_background, eyewitness_details=""):

    verdict = judge.verdict()

    return verdict  # Optionally return the verdict for further use

run_trial(case_background)

0

In [23]:
# df = pd.read_csv("cases.csv")

df4 = df.drop('Unnamed: 0', axis=1)

df4



,id,text
0,1989_75,civil appellate jurisdiction civil appeal numb...
1,1959_102,civil appellate jurisdiction civil appeal numb...
2,1979_76,civil appellate jurisdiction civil appeal numb...
3,1991_124,civil appellate jurisdiction civil appeal numb...
4,1985_233,original jurisdiction writ petitions number. 8...
...,...,...
95,1968_146,\nramaswami j. \n\nthese appeals are brought b...
96,1992_528,civil appellate jurisdiction special leave pet...
97,1958_35,criminal appellate jurisdiction criminal appea...
98,1987_407,civil appellate jurisdiction civil appeal numb...


In [ ]:
import pandas as pd

results = []

for idx, row in df.iterrows():
    case_background = row['text']          
    max_length = 40000 
    case_background = case_background[:max_length]
    eyewitness_details = ""

    judge = JudgeAgent(
        model=model,
        token=token,
        case_background=case_background,
        eyewitness_details=eyewitness_details
    )
    verdict = judge.verdict()  # 0 or 1

    print(verdict)

    results.append({'id': row['id'], 'text': case_background, 'verdict': verdict})

# Convert results to DataFrame
df_out = pd.DataFrame(results)

# Save to CSV
df_out[['id', 'text', 'verdict']].to_csv("judged_cases.csv", index=False)


1
0
1
0
1
1
1
1
0
0
0
1
1
0
0
0
1
0
1
0
0
0
1
0
1
0
0
0
1
0
0
0
0
0
1
1


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01js7ms5kyfe7swnf83rbg8b8z` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 98195, Requested 9874. Please try again in 1h56m11.423999999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}